In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage

import google.generativeai as genai

import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')



now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

In [12]:
gemini_result_kospi_20240115 = pd.read_csv('data_crawler/dashboard/gemini_result_kospi_20240115.csv', 
                                          dtype = {'ticker':str})

In [13]:
gemini_result_kospi_20240115

,ticker,response_msg
0,005930,**증권 보고서**\n\n**날짜:** 2024-01-05 00:00:00+00:0...
1,000660,종목명: SK하이닉스(000660)\n\n날짜: 2024-01-05 00:00:00...
2,373220,**주식 종목: LG에너지솔루션 (373220)**\n\n- 기준일: 2024년 1...
3,207940,**삼성바이오로직스 증권 보고서**\n\n**날짜:** 2024년 1월 5일\n\n...
4,005935,"**삼성전자우(005935)**의 주가 변동 추이를 살펴보면,\n\n* 240일 기..."
...,...,...
102,051900,**증권 리포트**\n**날짜:** 2024-01-15\n\n**종목:** LG생활...
103,034220,**종목:** LG디스플레이 (034220)\n\n**날짜:** 2024년 1월 1...
104,267250,**증권 보고서**\n\n**날짜:** 2024-01-15\n\n**종목:** HD...
105,251270,**증권 보고서**\n\n**날짜:** 2024년 1월 15일\n\n**종목정보:*...
